# Lesson 2: ReACT Framework with Real Tool Integration - SOLUTION

## Understanding ReACT with Actual Tools

This notebook demonstrates **authentic ReACT (Reasoning and Acting)** where LLMs:
1. **Think** about what tools they need
2. **Act** by calling real investigation tools
3. **Observe** tool results and continue reasoning

**Key Difference**: Unlike basic ReACT that just describes actions, this version **actually executes tools** and processes real results.

**Time Required**: 15-20 minutes  
**Focus**: Practical ReACT implementation with financial investigation tools

## Setup: Import Pre-Built Investigation Tools

In [ ]:
# Import standard libraries
import os
from dotenv import load_dotenv
from openai import OpenAI
import json

# Import our pre-built investigation tools
from investigation_tools import (
    get_transaction_history,
    get_customer_profile, 
    check_regulatory_thresholds,
    process_tool_calls
)

# Load environment and setup OpenAI client
load_dotenv('../../.env')
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✅ Setup complete - ReACT tools ready!")
print("Available tools: transaction history, customer profiles, regulatory checks")

## Quick Tool Demo: See What We're Working With

In [ ]:
# Quick demo of our investigation tools
print("🔍 TOOL DEMO - Transaction History:")
demo_transactions = get_transaction_history("high_risk_account_001", days=14)
print(json.dumps(demo_transactions, indent=2))

print("\n👤 TOOL DEMO - Customer Profile:")
demo_customer = get_customer_profile("CUST_001")
print(json.dumps(demo_customer, indent=2))

print("\n📋 TOOL DEMO - Regulatory Check:")
demo_check = check_regulatory_thresholds(9800, "cash_deposit")
print(json.dumps(demo_check, indent=2))

## ReACT Prompt: Teaching LLM to Use Tools

In [ ]:
# ReACT prompt that teaches the LLM to use our tools
REACT_PROMPT = """
You are a Financial Crimes Investigator using real investigation tools.

AVAILABLE TOOLS:
1. get_transaction_history(account_id, days) - Get account transactions
2. get_customer_profile(customer_id) - Get customer info and risk data
3. check_regulatory_thresholds(amount, type) - Check compliance requirements

PROCESS: Use THOUGHT → ACTION → OBSERVATION cycle:

THOUGHT: [Decide what you need to investigate]
ACTION: [Call tools using this exact JSON format]
```json
{
  "tool": "tool_name",
  "parameters": {"param1": "value1", "param2": "value2"}
}
```
OBSERVATION: [Analyze the tool results]

Continue until you have enough information for a final recommendation.
"""

print("📋 ReACT prompt configured for tool usage")

## Investigation Scenario: Suspicious Cash Deposits

In [ ]:
# Investigation case for ReACT analysis
investigation_case = """
SUSPICIOUS ACTIVITY ALERT

Case: Multiple cash deposits just under $10,000
Customer ID: CUST_001
Account ID: high_risk_account_001
Time Period: Past 14 days
Alert Source: Branch manager noticed pattern
Customer Explanation: "Restaurant business doing really well"

TASK: Investigate and determine if SAR filing is required.
"""

print("📁 Investigation scenario loaded")
print("Focus: Potential cash structuring pattern")

## ReACT Investigation in Action

In [ ]:
def run_react_investigation(case_details, max_rounds=3):
    """Run ReACT investigation with real tool integration"""
    
    print("🚀 STARTING ReACT INVESTIGATION")
    print("=" * 50)
    
    context = case_details
    
    for round_num in range(1, max_rounds + 1):
        print(f"\n🔄 ROUND {round_num}")
        print("-" * 30)
        
        # Get LLM reasoning and tool calls
        prompt = f"""{REACT_PROMPT}

CASE DETAILS:
{context}

Conduct your investigation using THOUGHT → ACTION → OBSERVATION.
"""
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
                max_tokens=600
            )
            
            llm_response = response.choices[0].message.content
            print("🤖 INVESTIGATOR RESPONSE:")
            print(llm_response)
            print()
            
            # Execute any tool calls
            tool_results = process_tool_calls(llm_response)
            
            if tool_results:
                # Add tool results to context for next round
                context += f"\n\nROUND {round_num} TOOL RESULTS:\n"
                for result in tool_results:
                    context += f"- {result['tool']}: {json.dumps(result['result'], indent=2)}\n"
            else:
                # No tools called - likely final conclusion
                print("✅ Investigation complete - no more tools needed")
                break
                
        except Exception as e:
            print(f"❌ Error in Round {round_num}: {str(e)}")
            break
    
    print("\n" + "=" * 50)
    print("🏁 INVESTIGATION COMPLETE")

# Run the investigation
run_react_investigation(investigation_case)

## Compare: Basic vs ReACT with Tools

Let's see the difference between basic analysis and ReACT with real tools:

In [ ]:
# Basic analysis without tools
print("📊 COMPARISON: Basic Analysis vs ReACT with Tools\n")

print("🔸 BASIC ANALYSIS (No Tools):")
basic_prompt = "Analyze this suspicious activity case and provide recommendations."

basic_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": f"{basic_prompt}\n\n{investigation_case}"}],
    temperature=0.3,
    max_tokens=300
)

print(basic_response.choices[0].message.content)
print("\n" + "="*60)
print("🔸 ReACT WITH TOOLS (Above):")
print("- Used real transaction data")
print("- Checked actual customer profile")
print("- Verified regulatory thresholds")
print("- Made data-driven decisions")
print("\n✅ ReACT with tools provides evidence-based investigation!")

## Test Your Own Scenario

In [ ]:
# Template for testing different scenarios
custom_scenario = """
INVESTIGATION CASE: [Your Case Title]

Case: [Description of suspicious activity]
Customer ID: [Choose CUST_001, CUST_002, or CUST_003]
Account ID: [Choose high_risk_account_001, business_account_002, or normal_account]
Alert Source: [How was this detected]
Customer Explanation: [What customer says]

TASK: [What you want the investigator to determine]
"""

print("💡 TESTING FRAMEWORK:")
print("1. Fill in the template above")
print("2. Run: run_react_investigation(your_scenario)")
print("3. Watch ReACT use tools to investigate")
print("\nAvailable test combinations:")
print("- CUST_001 + high_risk_account_001: Restaurant manager, cash deposits")
print("- CUST_002 + business_account_002: Business owner, wire transfers")
print("- CUST_003 + normal_account: Software engineer, normal transactions")

## Key Takeaways: Real ReACT Implementation

### What We Demonstrated:

**1. Authentic Tool Integration**
- ✅ Real function calls with actual parameters
- ✅ JSON parsing and tool execution
- ✅ Tool results fed back into reasoning

**2. Practical ReACT Workflow**
- ✅ LLM decides what tools to use based on investigation needs
- ✅ Tools provide real data for analysis
- ✅ Iterative refinement based on tool results

**3. Financial Investigation Benefits**
- ✅ Evidence-based decisions vs. speculation
- ✅ Systematic data gathering
- ✅ Regulatory compliance checking
- ✅ Audit trail of investigation steps

### vs. Basic ReACT (Simulated Actions):
- **Basic ReACT**: "I would check the transaction history" ❌
- **ReACT with Tools**: Actually retrieves and analyzes transaction data ✅

### Real-World Applications:
- **Financial Compliance**: Automated suspicious activity investigation
- **Risk Assessment**: Data-driven customer risk evaluation
- **Regulatory Reporting**: Evidence gathering for SAR/CTR filings
- **Case Management**: Structured investigation workflows

**The Power**: ReACT with real tools transforms LLMs from "thinking about actions" to "actually taking actions" with real data and measurable outcomes! 🚀

---
*Time: ~15-20 minutes | Focus: Practical ReACT Implementation | Context: Financial Investigation*